In [12]:
import re
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

### 这里我们使用滑动窗口进行数据采样来构建训练数据集

#### BPE读取数据

In [3]:
with open("the-verdict.txt", "r", encoding='utf-8') as f:
    raw_text = f.read()
tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [4]:
# 滑动窗口构建训练数据集
context_size = 4 # batch_size大小
x = enc_text[50:context_size + 50]
y = enc_text[51:context_size + 50]
print(f"x: {x}") 
print(f"y:       {y}")

x: [290, 4920, 2241, 287]
y:       [4920, 2241, 287]


In [8]:
for i in range(1, context_size+1): 
 context = enc_text[50:i + 50] 
 desired = enc_text[i + 50] 
 print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


### 实现高效数据加载器

#### 数据加载器会遍历输入数据集，并将输入和目标以 PyTorch 张量的形式返回，这些PyTorch 张量可以被视为多维数组。具体来说，我们的目标是返回两个张量：一个是包含大语言模型所见的文本输入的输入张量，另一个是包含大语言模型需要预测的目标词元的目标张量

In [19]:
# Dataset
class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [20]:
# DataLoader
def GPTDataLoader(txt, batch_size, max_length, stride, shuffle = True, 
                  drop_last = True, num_workers = 0, tokenizer = tokenizer):
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        num_workers=num_workers,
        shuffle=shuffle
    )
    return dataloader

In [22]:
with open("the-verdict.txt", "r", encoding="utf-8") as f: 
 raw_text = f.read() 
dataloader = GPTDataLoader( 
 raw_text, batch_size=2, max_length=4, stride=1, shuffle=False) 
data_iter = iter(dataloader) 
first_batch = next(data_iter) 
print(first_batch)

[tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619]])]
